In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

## load data

In [2]:
train_df = pd.read_csv("st_training_set.csv", index_col=0)
test_df = pd.read_csv("st_test_set.csv", index_col=0)

df = pd.concat((train_df, test_df))
train_mask = torch.arange(train_df.shape[0])
test_mask = torch.arange(train_df.shape[0], df.shape[0])

df

CHGA       PDL1      CD56       CK7     FoxP3      CD21      MUC1  \
0    -0.066992  -0.262392 -0.406045 -0.030104  0.000522 -0.318234 -0.144450   
1    -0.064515  -0.225181 -0.416845 -0.030104 -0.168616 -0.243178 -0.144450   
2    -0.059077   0.472951  1.011085 -0.030104 -0.169786  1.274733 -0.144450   
3    -0.066915  -0.262392  0.120701 -0.030104 -0.169786 -0.112518 -0.144450   
4    -0.062758  -0.262392 -0.496491 -0.030104 -0.057561 -0.078233 -0.144450   
...        ...        ...       ...       ...       ...       ...       ...   
1795  0.470625  -0.128228  1.184783 -0.030104 -0.169786 -0.224169 -0.144450   
1796 -0.063665   1.351185  2.800148 -0.030104 -0.169786 -0.221515 -0.144450   
1797  0.130528  -0.137102  1.291396 -0.030104 -0.169431  0.054145 -0.143911   
1798 -0.068577  10.208482  3.206462 -0.030104 -0.169786 -0.106464 -0.144450   
1799 -0.070000  -0.146833  0.008753 -0.029641 -0.168822 -0.163768 -0.143122   

           PD1      CD11b       CD4  ...  Arginase1       CD73     CD206  \
0    -0.337050   2.864545 -0.681588  ...   1.525496  12.999733 -0.249438   
1    -0.117195  -0.306023 -0.499291  ...  -0.293192  -0.471492 -0.243877   
2     0.781676  12.876779 -0.352096  ...   3.494604  -0.471492  1.262360   
3    -0.159795   4.298516 -0.637332  ...   3.851783  -0.452962 -0.173831   
4    -0.280044  -0.306013 -0.242123  ...  -0.363566   0.269699  2.070491   
...        ...        ...       ...  ...        ...        ...       ...   
1795 -0.109930  -0.306023 -0.679208  ...  -0.029983   2.249483 -0.233056   
1796 -0.350169  -0.306023 -0.666910  ...  -0.480942   0.139709 -0.245059   
1797  0.517339   0.931787  3.422995  ...   0.839774   0.843414 -0.182852   
1798 -0.133341  -0.306023 -0.470078  ...  -0.465993   1.049067 -0.236738   
1799 -0.189896  -0.250931 -0.622975  ...   0.001833  -0.087157 -0.218058   

          MUC6      COX2       MMP9       x       y         sample_name  \
0    -0.137942 -0.174521   0.210699  8674.0   471.0              tonsil   
1    -0.129458 -0.114048  -0.161118  7522.0  7301.0              tonsil   
2     0.067442  0.254756  15.389048  1162.0  1457.0              tonsil   
3    -0.128952 -0.146326  11.735550  6740.0  3583.0              tonsil   
4    -0.137942 -0.092463  -0.161118  3694.0  4966.0              tonsil   
...        ...       ...        ...     ...     ...                 ...   
1795 -0.137942 -0.171728  -0.161118   879.0  3010.0  Barretts Esophagus   
1796 -0.137942 -0.166028  -0.161118   949.0  3563.0  Barretts Esophagus   
1797  0.221214  0.011764   0.717192  8377.0  1125.0  Barretts Esophagus   
1798 -0.077550 -0.119248  -0.160221  1608.0  6112.0  Barretts Esophagus   
1799 -0.103163 -0.093328  -0.156764  8200.0  1586.0  Barretts Esophagus   

                cell_type  
0                    pdpn  
1     squamous_epithelial  
2                  innate  
3                  innate  
4                    pdpn  
...                   ...  
1795               stroma  
1796                nerve  
1797                    t  
1798                nerve  
1799         smoothmuscle  

[3552 rows x 48 columns]

## get edge

In [3]:
# 如果两个细胞的距离小于给定阈值，则在两个细胞间构边
def get_edge_index(pos, distance_thres=10):
    edge_list = []
    dists = pairwise_distances(pos)
    dists_mask = dists < distance_thres
    np.fill_diagonal(dists_mask, 0)
    edge_list = np.transpose(np.nonzero(dists_mask))
    return edge_list

pos = df.iloc[:, -4:-2].values
edge_index = get_edge_index(pos)

## make dataset

In [4]:
x = df.iloc[:, :-4].values
y = df['cell_type'].values

le = LabelEncoder()
y = le.fit_transform(y)

dataset = Data(x=torch.FloatTensor(x), 
                     edge_index=torch.LongTensor(edge_index).T, 
                     y=torch.LongTensor(y))

dataset.train_mask = train_mask
dataset.test_mask = test_mask

## make model

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    
model = GCN(dataset.x.shape[1], 32, np.unique(dataset.y).shape[0])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

## training

In [6]:
model.train()
for epoch in tqdm(range(300)):
    optimizer.zero_grad()
    out = model(dataset)
    loss = F.nll_loss(out[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()

100%|██████████| 300/300 [00:00<00:00, 305.84it/s]


## test

In [7]:
model.eval()
pred = model(dataset).argmax(dim=1)

report = classification_report(le.inverse_transform(dataset.y[dataset.test_mask]), le.inverse_transform(pred[dataset.test_mask]), output_dict=True)
report = pd.DataFrame(report).T
report

precision    recall  f1-score      support
endothelial           0.779412  0.795000  0.787129   200.000000
innate                0.922619  0.775000  0.842391   200.000000
nerve                 0.898618  0.975000  0.935252   200.000000
pdpn                  0.753138  0.900000  0.820046   200.000000
plasma                0.917241  0.665000  0.771014   200.000000
smoothmuscle          0.771318  0.995000  0.868996   200.000000
squamous_epithelial   0.683453  0.950000  0.794979   200.000000
stroma                0.709402  0.415000  0.523659   200.000000
t                     0.965517  0.840000  0.898396   200.000000
accuracy              0.812222  0.812222  0.812222     0.812222
macro avg             0.822302  0.812222  0.804651  1800.000000
weighted avg          0.822302  0.812222  0.804651  1800.000000